<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## Quick recognition example.
> **NOTE**: Before executing this example please clone **default.json.example** to **default.json** in ./config 

In [5]:
# Import all necessary libraries.
import os
import sys
import json
import matplotlib.image as mpimg


# change this property
NOMEROFF_NET_DIR = "/var/www/nomeroff-net/"
MASK_RCNN_DIR = "/var/www/Mask_RCNN/"

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, "logs/")
MASK_RCNN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/mask_rcnn_numberplate_0700.h5")
REGION_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/npregion_2018-12-27-295x64-2.h5")

sys.path.append(NOMEROFF_NET_DIR)

In [6]:
# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector,  RegionDetector, Detector, textPostprocessing

# Initialize npdetector with default configuration file.
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)

# Load weights in keras format.
nnet.loadModel(MASK_RCNN_MODEL_PATH)

# Initialize rect detector with default configuration file.
rectDetector = RectDetector()

# Initialize text detector.
textDetector = TextDetector()

# Initialize standart detector.
regionDetector = RegionDetector()
regionDetector.load(REGION_MODEL_PATH)

/usr/local/lib64/python3.6/site-packages/keras/engine/saving.py:304: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [7]:
# Detect numberplate
img_path = 'images/example1.jpeg'
img = mpimg.imread(img_path)
NP = nnet.detect([img])

/usr/local/lib64/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [8]:
# Generate image mask.
cv_img_masks = filters.cv_img_mask(NP)
for img_mask in cv_img_masks:
    # Detect points.
    points = rectDetector.detect(img_mask)
    # Split on zones
    zone = rectDetector.get_cv_zones(img, points)
    
    # find standart
    regionId = regionDetector.predict(zone)
    regionName = regionDetector.getLabels(regionId)

    # find text with postprocessing by standart
    text = textDetector.detect(zone)
    text = textPostprocessing(text, regionName)
    print(text)
    

AC4921CB
